<a href="https://colab.research.google.com/github/slafton/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-17 01:12:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.68MB/s    in 0.2s    

2022-10-17 01:12:50 (4.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [8]:
#Filter the vine_table to retrieve all votes where the total_votes >= to 20
total_votes_df = vine_df.filter(vine_df.total_votes >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [9]:
from pyspark.sql import functions as F

#Create new dataframe to retrieve all helpful_votes/total_votes >= 50%
total_votes_df = total_votes_df.withColumn("Percentage", F.col("helpful_votes")/F.col("total_votes")*100)
total_votes_df = total_votes_df.withColumn ("Percentage", F.round(F.col("Percentage"),2))

#total_votes_df.show()

helpful_votes_df = total_votes_df.filter(total_votes_df.Percentage >= 50)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|      80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|     67.74|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|     96.43|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|     100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|      87.3|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|     88.89|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|     95.83|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|      78.8|
|R3THZWOBS1086J|     

In [10]:
paid_df = helpful_votes_df. filter(helpful_votes_df.vine == "Y")
paid_df.show()

+---------+-----------+-------------+-----------+----+-----------------+----------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage|
+---------+-----------+-------------+-----------+----+-----------------+----------+
+---------+-----------+-------------+-----------+----+-----------------+----------+



In [11]:
unpaid_df = helpful_votes_df. filter(helpful_votes_df.vine == "N")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+----------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+----------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|      80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|     67.74|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|     96.43|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|     100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|      87.3|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|     88.89|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|     95.83|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|      78.8|
|R3THZWOBS1086J|     

In [12]:
#Determine total paid reviews
total_paid_reviews = paid_df.count()
print(f"Paid DataFrame review count: {total_paid_reviews}")

Paid DataFrame review count: 0


In [13]:
#Determine number of 5-star reviews for paid
total_five_paid = paid_df.where(paid_df.star_rating == '5').count()
print(f"Paid 5-star review count: {total_five_paid}")

Paid 5-star review count: 0


In [14]:
percentage_five_paid = round(total_five_paid/total_paid_reviews * 100, 2)
print(f"Paid 5-star review percentage: {percentage_five_paid}%")

ZeroDivisionError: ignored

In [15]:
#Determine total unpaid reviews
total_unpaid_reviews = unpaid_df.count()
print(f"Unpaid DataFrame review count: {total_unpaid_reviews}")

Unpaid DataFrame review count: 403807


In [16]:
#Determine number of 5-star reviews for unpaid
total_five_unpaid = unpaid_df.where(unpaid_df.star_rating == '5').count()
print(f"Unpaid 5-star review count: {total_five_unpaid}")

Unpaid 5-star review count: 242889


In [17]:
percentage_five_unpaid = round(total_five_unpaid/total_unpaid_reviews * 100, 2)
print(f"Unpaid 5-star review percentage: {percentage_five_unpaid}%")


Unpaid 5-star review percentage: 60.15%
